In [7]:
import sqlite3

In [8]:
conn = sqlite3.connect('db/knowledge_graph.db')

In [9]:
cursor = conn.cursor()

In [11]:
cursor.execute('SELECT COUNT(*) FROM claims')
cursor.fetchone()

(0,)

---

In [4]:
cursor.execute('ALTER TABLE properties DROP COLUMN property_description')

OperationalError: no such column: "property_description"

In [ ]:
conn.commit()

In [ ]:
cursor.execute('''
CREATE TABLE items (
    item_id TEXT PRIMARY KEY,
    item_alias TEXT NOT NULL,
    item_description TEXT
)
''')

In [ ]:
cursor.execute('''
CREATE TABLE properties (
    property_id TEXT PRIMARY KEY,
    property_alias TEXT NOT NULL,
    property_description TEXT
)
''')

In [5]:
cursor.execute('''
CREATE TABLE claims (
    claim_id INTEGER PRIMARY KEY,
    subject_id TEXT,
    property_id TEXT,
    target_id TEXT,
    FOREIGN KEY (subject_id) REFERENCES items (item_id),
    FOREIGN KEY (property_id) REFERENCES properties (property_id),
    FOREIGN KEY (target_id) REFERENCES items (item_id)
)
''')

OperationalError: table claims already exists

In [6]:
cursor.execute('SELECT * FROM claims')
all_claims = cursor.fetchall()
print(all_claims)

[]


In [5]:
cursor.execute('''
    INSERT INTO claims (subject_id, property_id, target_id)
    VALUES (?, ?, ?)
''', ('Q42', 'P1', 'Q2'))
conn.commit()

In [6]:
cursor.execute('''
    DELETE FROM claims
    WHERE claim_id = ?
''', (1,))

conn.commit()

In [7]:
item_alias = {}
with open('alias/wikidata5m_entity.txt', 'r') as f:
    for item in f:
        data = item[:-1].split('\t')
        item_alias[data[0]] = ', '.join(data[1:])

In [8]:
prop_alias = {}
with open('alias/wikidata5m_relation.txt', 'r') as f:
    for item in f:
        data = item[:-1].split('\t')
        prop_alias[data[0]] = ', '.join(data[1:])

In [9]:
text_desc = {}
with open('wikidata5m_text.txt', 'r') as f:
    for item in f:
        data = item[:-1].split('\t')
        text_desc[data[0]] = '\t'.join(data[1:])

In [26]:
for id, alias in item_alias.items():
    try:
        if id in text_desc:
            desc = text_desc[id]
        else:
            desc = ''
        cursor.execute('''
            INSERT INTO items (item_id, item_alias, item_description)
            VALUES (?, ?, ?)
        ''', (id, alias, desc))
    except Exception as e:
        print(id, alias, desc)
        raise e

conn.commit()

In [23]:
cursor.execute('DELETE FROM items')
conn.commit()

In [29]:
cursor.execute('SELECT COUNT(*) FROM properties')
cursor.fetchone()[0]

825

In [28]:
for id, alias in prop_alias.items():
    try:
        if id in text_desc:
            desc = text_desc[id]
        else:
            desc = ''
        cursor.execute('''
            INSERT INTO properties (property_id, property_alias, property_description)
            VALUES (?, ?, ?)
        ''', (id, alias, desc))
    except Exception as e:
        print(id, alias, desc)
        raise e

conn.commit()

In [13]:
with open('data/wikidata5m_transductive_train.txt', 'r') as f:
    for i, claim in enumerate(f):
        subject_id, property_id, target_id = claim[:-1].split('\t')
        cursor.execute('''
            INSERT INTO claims (subject_id, property_id, target_id)
            VALUES (?, ?, ?)
        ''', (subject_id, property_id, target_id))

In [15]:
cursor.execute('SELECT COUNT(*) FROM claims')
cursor.fetchone()[0]

20614279

In [16]:
conn.commit()

In [35]:
conn.close()

In [14]:
target_item_id = 'Q6992836'  # Make sure this item_id exists in your 'items' table

cursor.execute('''
SELECT COUNT(*) FROM claims
WHERE target_id = ?
''', (target_item_id,))
count_result = cursor.fetchone()[0]
print(f"Number of properties pointing to item {target_item_id}: {count_result}")


Number of properties pointing to item Q6992836: 0
